In [0]:
import pandas as pd
import requests
import os

DATA_PATH = "/Volumes/workspace/hackathon/data/"
GEO_JSON_FN = "sydney.geojson"
POSTCODES_CSV_FN = "australian_postcodes.csv"

# Download sydney geojson
geo_json_path = os.path.join(DATA_PATH, GEO_JSON_FN)
if not os.path.exists(geo_json_path):
    geo_url = "https://raw.githubusercontent.com/codeforamerica/click_that_hood/master/public/data/sydney.geojson"
    response = requests.get(geo_url)
    with open(os.path.join(DATA_PATH, GEO_JSON_FN), "w") as f:
        f.write(response.text)
    print("GEO written.")


In [0]:
import json
# Load GeoJSON
with open("/Volumes/workspace/hackathon/data/sydney.geojson") as f:
    geojson = json.load(f)

# print(type(geojson))
# for feat in geojson["features"]:
#     print(feat)

In [0]:
# load suburb data
# filter: 1001 to 4385 is NSW
suburb_df = pd.read_csv(os.path.join(DATA_PATH, POSTCODES_CSV_FN))
nsw_suburb_df = suburb_df.loc[(suburb_df["postcode"]>=1001)&(suburb_df["postcode"]<=4385)]

In [0]:
import plotly.express as px
import pandas as pd

# generate dummy data
# dummy data: comes from chatgpt - make some dummy value and plot on the map
data = {
    "postcode": ["2000", "2001", "2007", "2010", "2020"],
    "value": [12000, 8000, 15000, 9000, 11000]
}
df = pd.DataFrame(data)

# Ensure both postcode columns are of the same type
df['postcode'] = df['postcode'].astype(int)
nsw_suburb_df['postcode'] = nsw_suburb_df['postcode'].astype(int)

# Merge data with locality information
# merged_df = df.merge(nsw_suburb_df.loc[:, ['postcode', 'locality']], on='postcode', how='left')
merged_df = pd.merge(
    left=df
    ,right=nsw_suburb_df.loc[:, ["postcode", "locality"]]
    ,on="postcode"
    ,how="left"
)


# # Check the properties to match postcodes
# print(geojson['features'][0]['properties'])  # usually has 'name' or 'postcode'

display(merged_df)


In [0]:
fig = px.choropleth(
    df,
    geojson=geojson,
    locations="postcode",
    color="population",
    color_continuous_scale="Viridis",
    title="Sydney Population by Postcode"
)

fig.update_geos(fitbounds="locations", visible=False)
fig.update_layout(height=600, margin={"r":0,"t":30,"l":0,"b":0})
fig.show()